In [312]:
# !pip install pyarrow

In [313]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

#import xgbtune 

In [314]:
df = pd.read_parquet("../data/dataset.parquet")
df

,date,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,...,volume_lag_25,volume_lag_26,volume_lag_27,volume_lag_28,volume_lag_29,day_of_week,day_of_month,month,year,week_of_year
0,2004-08-10,0.542679,0.563214,0.541964,0.562857,0.475869,351036000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,10,8,2004,33
1,2004-08-11,0.555357,0.555893,0.540357,0.553750,0.468169,322392000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,11,8,2004,33
2,2004-08-12,0.543750,0.550893,0.540714,0.542321,0.458507,226200800,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3,12,8,2004,33
3,2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465603,328048000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
4,2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464697,435674400,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,2024-08-02,219.149994,225.600006,217.710007,219.860001,219.860001,105568600,0,2.394394,0.015493,...,49772700.0,66213200.0,56713900.0,80727000.0,246421400.0,4,2,8,2024,31
5030,2024-08-05,199.089996,213.500000,196.000000,209.270004,209.270004,119548600,0,8.237186,0.036997,...,82542700.0,49772700.0,66213200.0,56713900.0,80727000.0,0,5,8,2024,32
5031,2024-08-06,205.300003,209.990005,201.070007,207.229996,207.229996,69660500,1,9.265285,0.040549,...,60402900.0,82542700.0,49772700.0,66213200.0,56713900.0,1,6,8,2024,32
5032,2024-08-07,206.899994,213.639999,206.389999,209.820007,209.820007,63516400,1,9.693821,0.041003,...,58046200.0,60402900.0,82542700.0,49772700.0,66213200.0,2,7,8,2024,32


In [315]:
# target = "target_close"
# target = "close_percent_change_1"
target = "up_down_close_1"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

,up_down_close_1
date,
2004-08-10,0
2004-08-11,0
2004-08-12,1
2004-08-13,0
2004-08-16,1
...,...
2024-08-02,0
2024-08-05,0
2024-08-06,1


In [316]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

,Column Name,Data Type
0,volume,int64
0,on_balance_volume_1,int64
0,dividends_event_name,object
0,splits_event_name,object


In [317]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64 (XGBoosted models cannot use strings, but categories as enumerated values)
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

Dropped Column: dividends_event_name
Dropped Column: splits_event_name


In [318]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [319]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# x_train

In [320]:
# y_test

In [321]:
x_train

,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,high_close_7,...,volume_lag_25,volume_lag_26,volume_lag_27,volume_lag_28,volume_lag_29,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.416982,3.022208e+08,0.147911,0.007639,0.125000,0.267858,...,254760800.0,4.282712e+08,4.090212e+08,447980400.0,371938000.0,1,3,1,2012,1
2020-01-23,79.480003,79.889999,78.912498,79.807503,77.546753,1.044720e+08,0.606827,0.009795,0.977501,0.464996,...,128186000.0,1.335876e+08,1.373104e+08,78756800.0,90420400.0,3,23,1,2020,4
2017-04-21,35.610001,35.669998,35.462502,35.567501,33.157009,6.928360e+07,0.101897,0.005885,0.207497,0.059998,...,76928000.0,1.027672e+08,6.123640e+07,69686800.0,78451200.0,4,21,4,2017,16
2020-09-24,105.169998,110.250000,105.000000,108.220001,105.879845,1.677433e+08,3.883301,0.047581,5.250000,3.129997,...,145538000.0,1.056336e+08,1.195616e+08,165565200.0,210082000.0,3,24,9,2020,39
2007-03-09,3.171429,3.173214,3.121429,3.141786,2.656232,4.518360e+08,0.060982,0.014121,0.051785,0.030357,...,664342000.0,8.560692e+08,5.779704e+08,901664400.0,986874000.0,4,9,3,2007,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-01-28,4.577143,4.757143,4.516071,4.643214,3.925617,1.474844e+09,0.457265,0.045689,0.241072,0.113929,...,827478400.0,1.222603e+09,1.024694e+09,674312800.0,864617600.0,0,28,1,2008,5
2017-09-28,38.472500,38.570000,38.174999,38.320000,36.011284,8.802200e+07,0.677053,0.014575,0.395000,0.012501,...,77596400.0,8.641840e+07,1.054740e+08,109712400.0,111762400.0,3,28,9,2017,39
2014-02-26,18.700357,18.750000,18.414286,18.476786,16.252596,2.762172e+08,0.327653,0.012440,0.335714,0.105000,...,328526800.0,4.267396e+08,2.292780e+08,391638800.0,332561600.0,2,26,2,2014,9


In [322]:
import numpy

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x< stop, count(start, step))

# list(frange(0, 1, 0.1))
numpy.linspace(5, 10, num=6).tolist()

[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [323]:
y_train

,up_down_close_1
date,
2012-01-03,1
2020-01-23,0
2017-04-21,1
2020-09-24,1
2007-03-09,1
...,...
2008-01-28,1
2017-09-28,1
2014-02-26,1


In [324]:
from xgboost import XGBClassifier

# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.05, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.05, subsample=1, colsampvle_bytree=.3)
model = XGBClassifier(n_estimators=500, max_depth=5, eta=0.05, subsample=0.9, colsample_bytree=0.4)
model.fit(x_train, y_train)



pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

In [325]:
# print(model.best_params_)
# model


In [326]:
predicted = model.predict(x_test)
y_test["predicted"] = predicted
y_test

,up_down_close_1,predicted
date,,
2008-02-20,0,1
2018-07-19,0,1
2017-05-10,1,1
2023-05-30,0,1
2023-10-23,1,1
...,...,...
2022-05-31,0,1
2008-01-09,0,0
2010-06-29,0,1


In [327]:
# Classification accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test[target], y_test["predicted"])
print("accuracy:", accuracy)

# Classification recall
from sklearn.metrics import recall_score
recall = recall_score(y_test[target], y_test["predicted"])
print("recall:", recall)

# Classification precision
from sklearn.metrics import precision_score
precision = precision_score(y_test[target], y_test["predicted"])
print("precision:", precision)

# Classification f1
from sklearn.metrics import f1_score
f1 = f1_score(y_test[target], y_test["predicted"])
print("f1:", f1)

accuracy: 0.5213505461767627
recall: 0.6082677165354331
precision: 0.5219594594594594
f1: 0.5618181818181818


accuracy: 0.5153922542204568
recall: 0.6316793893129771
precision: 0.5287539936102237
f1: 0.5756521739130435

In [328]:
# Classification report
from sklearn.metrics import classification_report
print(classification_report(y_test[target], y_test["predicted"]))
# print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.52      0.43      0.47       499
           1       0.52      0.61      0.56       508

    accuracy                           0.52      1007
   macro avg       0.52      0.52      0.52      1007
weighted avg       0.52      0.52      0.52      1007



In [329]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test[target], y_test["predicted"])


array([[216, 283],
       [199, 309]], dtype=int64)

In [330]:
# Feature importance
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
# indices
# Print the feature ranking
print("Feature ranking:")
for f in range(x_train.shape[1]):
    print(f"{f + 1}. feature {x_train.columns[indices[f]]} ({importances[indices[f]]})")

Feature ranking:
1. feature low_rolling_mean_30 (0.008248868398368359)
2. feature close_fib_0.236 (0.0070532276295125484)
3. feature high_rolling_mean_7 (0.006506121717393398)
4. feature close_lag_19 (0.005871776957064867)
5. feature low_rolling_exp_mean_30 (0.0058556850999593735)
6. feature high_bollinger_upper_14 (0.00569919915869832)
7. feature low_lag_25 (0.00555780716240406)
8. feature high_lag_24 (0.005515784490853548)
9. feature low_lag_16 (0.0054850270971655846)
10. feature open_lag_23 (0.005313495174050331)
11. feature close_lag_9 (0.005312545690685511)
12. feature volitility_low_30 (0.005047682207077742)
13. feature close_lag_21 (0.005035256966948509)
14. feature high_lag_25 (0.005020950920879841)
15. feature high_lag_7 (0.004902285989373922)
16. feature low_lag_24 (0.004848625510931015)
17. feature low_fib_0.382 (0.004789348691701889)
18. feature low_lag_19 (0.004764693323522806)
19. feature open_lag_16 (0.0047082151286304)
20. feature high_lag_19 (0.004689421504735947)
21. 

In [333]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")